In [174]:
from loguru import logger
import numpy as np
from numpy.linalg import norm
from scipy.sparse import diags, csc_matrix
from time import time
import sys

In [175]:
logger_handlers = [
    {
        "sink": sys.stdout,
        "level": "INFO",
        "format": "<level>level={level} {message}</level>",
    }
]
logger.configure(handlers=logger_handlers)
args = {}
args["input"] = "matrix2.txt"
args["output"] = "output.txt"
args["eps"] = 0.01
args["diag"] = False

In [176]:
def read_matrix(filename: str):
    with open(filename) as f:
        shape = int(f.readline())
        matrix = [[float(num) for num in line.split()]
                  for _, line in zip(range(shape), f)]
        matrix = csc_matrix(matrix)
        b = np.array([float(num) for num in f.readline().split()])
        return matrix, b


In [177]:
class Solver:
    def __init__(self, matrix, b, output_file,
                 x0=None, eps=1e-5):
        self.output = 'res_default' if output_file is None else output_file
        self.matrix = matrix
        self.b = b
        self.eps = eps
        self.shape = matrix.shape[0]
        self.x0 = np.array([0] * self.shape) if x0 is None else x0
        self.k = 0



    def solve(self, max_iter=100000):
        x0 = self.x0
        r0 = self.b - self.matrix.dot(x0)
        p0 = np.copy(r0)
        for _ in range(max_iter):
            temp = self.matrix @ p0
            norm_0 = np.dot(r0, r0)
            alpha_i = norm_0 / (temp @ p0)
            x_new = x0 + p0 * alpha_i
            r_new = r0 - temp * alpha_i
            norm_new = r_new @ r_new
            beta_i = norm_new/norm_0
            p_new = r_new +  p0*beta_i

            r0 = r_new
            p0 = p_new
            x0 = x_new

            self.k+=1
            if norm(r_new) < self.eps:
                break
        return x0




    def solve_and_print(self):
        start = time()
        x = self.solve()
        end = time()
        start2 = time()
        x2 = np.linalg.solve(self.matrix.toarray(), self.b)
        end2 = time()
        logger.info('Custom solution:\n')
        logger.info(f'{x.round(5)}\n')
        logger.info(f'eps={self.eps} shape={self.shape} iterations={self.k} mean={np.mean(x)} time={round(end - start, 5)} seconds\n')
        logger.info('NumPy solution:\n')
        logger.info(f'{x2.round(5)}\n')
        logger.info(f'mean={np.mean(x2)} time={round(end2 - start2, 5)} seconds\n')


In [178]:
def main():

    matrix, b = read_matrix(args["input"])
    solver = Solver(matrix, b, output_file=args["output"], eps=args["eps"])
    solver.solve_and_print()


if __name__ == "__main__":
    main()

level=INFO Custom solution:

level=INFO [   0.4867  -139.21825   36.03465   40.9907   160.46605  -17.3655
   23.0181    26.00615   -8.50406  -41.04431]

level=INFO eps=0.01 shape=10 iterations=10 mean=8.087021967322894 time=0.001 seconds

level=INFO NumPy solution:

level=INFO [   0.4867  -139.21825   36.03465   40.9907   160.46605  -17.3655
   23.0181    26.00615   -8.50406  -41.04431]

level=INFO mean=8.08702196756482 time=0.0 seconds

